# Preliminares

In [1]:
!pip install opencv-python

In [2]:
!pip install opencv-contrib-python

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import string
import pandas as pd
import glob
import random 
from skimage.color import rgb2gray
import seaborn as sns
import scipy.integrate as spi
import math 
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
import sklearn

In [5]:
def spline3(A):     #spline cubico para la lista de coordenadas A 
    n = len(A) 
    a = []; h = []; x = []; y = []*(n-1); C = [0]*n
    alpha = []
    l = [1]
    B = [0] 
    g = [0]; gn = 0
    spline = []
    for i in range(n):
        a.append(A[i][1])
    for i in range(n-1):
        xh = A[i+1][0]-A[i][0]
        h.append(xh)
    for i in range(1, n-1):
        xa = (3/h[i])*(a[i+1]-a[i])-(3/h[i-1])*(a[i]-a[i-1])
        alpha.append(xa)
    for i in range(1, n-1):
        xl = 2*(A[i+1][0]-A[i-1][0])-h[i-1]*B[i-1]
        l.append(xl)
        xb = h[i]/l[i]
        B.append(xb)
        xg = (alpha[i-1]-h[i-1]*g[i-1])/l[i]
        g.append(xg)
    l.append(1)
    g.append(0)
    for i in range(n-1):
        j = (n-1)-(i+1)
        xC = g[j]-B[j]*C[j+1]
        C[j] = xC
        xy = ((a[j+1]-a[j])/h[j])-(h[j]/3)*(C[j+1]+2*C[j])
        y.append(xy)
        xx = (C[j+1]-C[j])/(3*h[j])
        x.append(xx)
    for i in range(n-1):
        j=(n-1)-(i+1)
        S3 = [a[i],y[j],C[i],x[j]]
        spline.append(S3)
    SC=np.array(spline)
    return SC 

In [6]:
def graficas(variables, etiquetas, colores, title, limit=True):
  if len(etiquetas)!=len(variables):
    print("La cantidad de etiquetas debe ser igual a la cantidad de variables")
  elif len(colores)!=len(variables):
    print("La cantidad de colores debe ser igual a la cantidad de variables")
  else:
    for j in range(len(variables)):
      A=variables[j]
      cond=[]            
      funcion=[]
      SP=spline3(A)
      cond=[]
      funcion=[]
      x=[]
      y=[]
      for i in range(len(spline3(A))):
          xa=np.linspace(A[i,0],A[i+1,0]-0.0001,11)
          x=np.concatenate((x,xa))
          ya=SP[i,0]+SP[i,1]*(xa-A[i,0])+SP[i,2]*(xa-A[i,0])**2+SP[i,3]*(xa-A[i,0])**3
          y=np.concatenate((y,ya))
      plt.plot(x,y,c=colores[j],label=etiquetas[j])
  if limit==True:
    plt.plot(x, x**0, 'k--')
    plt.ylim(0,1.05)
  plt.title(title)
  plt.xlabel("Time")
  plt.legend(loc=0)

  plt.show()

In [7]:
#---El color---

def color(A):                 #Transformación que permite visualizar el sistema en escala nipy_spectral
    n,m=A.shape               #dim(A)=nm
    C=np.zeros((n,m))         #Se realizará una copia de A para evitar problemas de apuntadores
    for i in range(n):          
        for j in range(m):
            C[i][j]=A[i][j]
    for i in range(n):
        for j in range(m):
            if C[i][j]==0:    #A los individuos susceptibles se les asigna el color amarillo
                C[i][j]=190   
            if C[i][j]==1:    #A los individuos infectados se les asigna el color rojo
                C[i][j]=240
            if C[i][j]==2:    #A los individuos recuperados se les asigna el color verde
                C[i][j]=115
            if C[i][j]==-1:   #A los espacios vacios se les asigna el color negro
                C[i][j]=0
            if C[i][j]==3:
                C[i][j]=256
    D=np.zeros((n+2,m+2)) 
    for i in range(n):                              
        for j in range(m):
            D[i+1][j+1]=C[i][j]
    D[0][0]=0; D[n+1][m+1]=256  #Polos de color para poder ajustar la escala de manera adecuada
    return D

# Definición de topologías

In [8]:
# Vecindad de Moore
def Moore(K,i,j):  
    A = np.zeros((3,3))      
    A[0][0] = K[i-1][j-1]    
    A[0][1] = K[i-1][j]      
    A[0][2] = K[i-1][j+1]    
    A[1][0] = K[i][j-1]      
    A[1][1] = K[i][j]        
    A[1][2] = K[i][j+1]      
    A[2][0] = K[i+1][j-1]    
    A[2][1] = K[i+1][j]      
    A[2][2] = K[i+1][j+1]    
    return [A,[1,1]]   #Los valores en su vecindad, junto con la coordenada de la célula principal

# Vecindad de Von Neumann
def Von_Neumann(K,i,j):  
    A = -np.ones((3,3))              
    A[0][1] = K[i-1][j]      
    A[1][0] = K[i][j-1]      
    A[1][1] = K[i][j]        
    A[1][2] = K[i][j+1]      
    A[2][1] = K[i+1][j]      
    return [A,[1,1]]  #Los valores en su vecindad, junto con la coordenada de la célula principal

# Reconoce a function como la vecindad del agente en la posición ij en el sistema K
def identificador(function,K,i,j): 
  v=function(K,i,j)[0]
  cc=[function(K,i,j)[1]]
  return [v,cc]

In [9]:
# Vecindad de Moore extendida
def Mooreextendida(K,i,j):  
    A = np.zeros((5,5))      
    A[0][0] = K[i-2][j-2]    
    A[0][1] = K[i-2][j-1]
    A[0][2] = K[i-2][j]      
    A[0][3] = K[i-2][j+1]
    A[0][4] = K[i-2][j+2]    
    A[1][0] = K[i-1][j-2]    
    A[1][1] = K[i-1][j-1]
    A[1][2] = K[i-1][j]      
    A[1][3] = K[i-1][j+1]
    A[1][4] = K[i-1][j+2]
    A[2][0] = K[i][j-2]    
    A[2][1] = K[i][j-1]
    A[2][2] = K[i][j]      
    A[2][3] = K[i][j+1]
    A[2][4] = K[i][j+2]
    A[3][0] = K[i+1][j-2]    
    A[3][1] = K[i+1][j-1]
    A[3][2] = K[i+1][j]      
    A[3][3] = K[i+1][j+1]
    A[3][4] = K[i+1][j+2]
    A[4][0] = K[i+2][j-2]    
    A[4][1] = K[i+2][j-1]
    A[4][2] = K[i+2][j]      
    A[4][3] = K[i+2][j+1]
    A[4][4] = K[i+2][j+2]
    return [A,[2,2]]   #Los valores en su vecindad, junto con la coordenada de la célula principal

In [10]:
system = np.zeros((5,5))
system

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [11]:
# Matriz indicador y vecindad aumentada

def complete(function,system,i,j,nNewRows,nNewColumns):
  neighborhood = function(system,i,j)[0]
  n,m = neighborhood.shape
  newNeighborhood = -np.ones((nNewRows,nNewColumns))
  emptyRows = int((nNewRows-n)/2)
  emptyColumns = int((nNewColumns-m)/2)
  for row in range(emptyRows,n+emptyRows):
    for column in range(emptyColumns,m+emptyColumns):
      newNeighborhood[row][column] = neighborhood[row-emptyRows][column-emptyColumns]
  return [newNeighborhood, [i+emptyRows,j+emptyColumns]]

complete(Von_Neumann,system,1,1,3,3)

[array([[-1.,  0., -1.],
        [ 0.,  0.,  0.],
        [-1.,  0., -1.]]), [1, 1]]

In [12]:
def numericalIndex(system):
  k = 0
  rows, columns = system.shape
  numericalMatrix = np.zeros((rows,columns))
  for r in range(rows):
    for c in range(columns):
      numericalMatrix[r][c] = k
      k += 1
  return numericalMatrix

numericalIndex(system)

array([[ 0.,  1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14.],
       [15., 16., 17., 18., 19.],
       [20., 21., 22., 23., 24.]])

In [13]:
def neighborhoodOverlay(neighborhoodGrade1,neighborhoodGrade2,system,i,j):
  a,b = system.shape
  systemInij = -np.ones((a,b))
  firstNeighborhood = neighborhoodGrade1(system,i,j)
  secondNeighborhood = neighborhoodGrade2(system,i,j)
  n1, m1 = firstNeighborhood[0].shape   # Dimensión de la primera vecindad
  n2, m2 = secondNeighborhood[0].shape   # Dimensión de la segunda vecindad
  n = max(n1,n2); m = max(m1,m2)  # Tamaño de la vecindad
  if n1 != n or m1 != m:
    firstComplete = complete(firstNeighborhood[0],firstNeighborhood[1][0],
                                    firstNeighborhood[1][1],n,m)
    firstNeighborhood[0] = firstComplete[0]
    firstNeighborhood[1] = firstComplete[1]
  if n2 != n or m2 != m:
    secondComplete = complete(secondNeighborhood[0],secondNeighborhood[1][0],
                                    secondNeighborhood[1][1],n,m)
    secondNeighborhood[0] = secondComplete[0]
    secondNeighborhood[1] = secondComplete[1]
  neighborhood = -np.ones((n,m))
  indicator = -np.ones((n,m))
  neighborsGrade1 = []
  if i != 0 and i != n and j != 0 and j != m:
    x = -1
    for r in range(n):
      y = -1
      for c in range(m):
        systemInij[i+x][j+y] = system[i+x][j+y]
        y += 1
      x += 1
  positions = []
  for r in range(a):
    for c in range(b):
      if systemInij[r][c] != -1:
        positions.append([r,c])
  for row in range(n):
    for column in range(m):
      if firstNeighborhood[0][row][column] != -1:
        neighborhood[row][column] = firstNeighborhood[0][row][column]
        indicator[row][column] = 1
        neighborsGrade1.append([row,column])
      elif secondNeighborhood[0][row][column] != -1 and firstNeighborhood[0][row][column] == -1:
        neighborhood[row][column] = secondNeighborhood[0][row][column]
        indicator[row][column] = 2
        neighborsGrade1.append([row,column])
      else:
        neighborhood[row][column] = -1
        indicator[row][column] = -1
        neighborsGrade1.append([row,column])
  indicatorPositions = []
  for p in range(len(neighborsGrade1)):
    if indicator[neighborsGrade1[p][0]][neighborsGrade1[p][1]] == 1:
      indicatorPositions.append(positions[p])
  return [neighborhood,indicator,indicatorPositions]

neighborhoodOverlay(Von_Neumann,Moore,system,2,2)

[array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]), array([[2., 1., 2.],
        [1., 1., 1.],
        [2., 1., 2.]]), [[1, 2], [2, 1], [2, 2], [2, 3], [3, 2]]]

In [14]:
system

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [16]:
def numericalIndicators(numericalSystem,indicators):
  numericalPositions = []
  for indicatorPosition in indicators:
    numericalPositions.append(int(numericalSystem[indicatorPosition[0]][indicatorPosition[1]]))
  return numericalPositions

numericalIndicators(numericalIndex(system),neighborhoodOverlay(Von_Neumann,Moore,system,3,2))

IndexError: ignored

In [ ]:
def incidenceMatrix(system,neighborhoodGrade1,neighborhoodGrade2,p,q):
    n,m = system.shape                           
    extendedSystem = -np.ones((n+(p*2),m+(q*2)))
    extendedNumericalSystem = -np.ones((n+(p*2),m+(q*2)))
    numericalSystem = numericalIndex(system)
    for i in range(n):                              
        for j in range(m):
            extendedSystem[i+p][j+q] = system[i][j]
            extendedNumericalSystem[i+p][j+q] = numericalSystem[i][j]
    print(extendedNumericalSystem)
    incidence = np.zeros((n*m,n*m))
    for i in range(n*m):
      k = 0
      for j in range(n*m):
        incidence[i][j]=k
        k += 1
    indicators = []
    for i in range(n):
      for j in range(m):
        neighbors = neighborhoodOverlay(neighborhoodGrade1,neighborhoodGrade2,extendedSystem,i+p,j+q)[2]
        print(neighbors)
        #print(numericalIndicators(extendedNumericalSystem,neighbors))

incidenceMatrix(system,Von_Neumann,Moore,1,1)

[[-1. -1. -1. -1. -1. -1. -1.]
 [-1.  0.  1.  2.  3.  4. -1.]
 [-1.  5.  6.  7.  8.  9. -1.]
 [-1. 10. 11. 12. 13. 14. -1.]
 [-1. 15. 16. 17. 18. 19. -1.]
 [-1. 20. 21. 22. 23. 24. -1.]
 [-1. -1. -1. -1. -1. -1. -1.]]
[[1, 1], [1, 2], [2, 1]]
[[1, 0], [1, 1], [1, 2], [2, 1]]
[[1, 0], [1, 1], [1, 2], [2, 1]]
[[1, 0], [1, 1], [1, 2], [2, 1]]
[[1, 0], [1, 1], [2, 1]]
[[0, 1], [1, 1], [1, 2], [2, 1]]
[[0, 1], [1, 0], [1, 1], [1, 2], [2, 1]]
[[0, 1], [1, 0], [1, 1], [1, 2], [2, 1]]
[[0, 1], [1, 0], [1, 1], [1, 2], [2, 1]]
[[0, 1], [1, 0], [1, 1], [2, 1]]
[[0, 1], [1, 1], [1, 2], [2, 1]]
[[0, 1], [1, 0], [1, 1], [1, 2], [2, 1]]
[[0, 1], [1, 0], [1, 1], [1, 2], [2, 1]]
[[0, 1], [1, 0], [1, 1], [1, 2], [2, 1]]
[[0, 1], [1, 0], [1, 1], [2, 1]]
[[0, 1], [1, 1], [1, 2], [2, 1]]
[[0, 1], [1, 0], [1, 1], [1, 2], [2, 1]]
[[0, 1], [1, 0], [1, 1], [1, 2], [2, 1]]
[[0, 1], [1, 0], [1, 1], [1, 2], [2, 1]]
[[0, 1], [1, 0], [1, 1], [2, 1]]
[[0, 1], [1, 1], [1, 2]]
[[0, 1], [1, 0], [1, 1], [1, 2]]
[[0, 1],

In [ ]:
system

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [ ]:
numericalIndex(system)

array([[ 0.,  1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14.],
       [15., 16., 17., 18., 19.],
       [20., 21., 22., 23., 24.]])

La relación de superposición no es conmutativa

In [ ]:
###########
  elif i == 0 and j != 0 and j != m:
    x = 0
    for r in range(n - 1):
      y = -1
      for c in range(m):
        systemInij[i+x][j+y] = system[i+x][j+y]
        y += 1
      x += 1

########

In [ ]:
##########################################################################################

In [ ]:
# Funciones de contadoras

# Vecindad
def sumaS(V,a,b):             #Cantidad de individuos susceptibles en la vecindad V
    sumaS=0                   #Antes de realizar el conteo se toma un contador igual a cero
    n,m=V.shape
    for i in range(n):        #Tres filas en la vecindad de Moore
        for j in range(m):    #Tres columnas en la vecindad de Moore
            if V[i][j]==0:    #Para cada individuo se verifica si está identificado con el estado S
                sumaS=sumaS+1 #Si es verdadero, aumente en uno el contador de susceptibles
    if V[a][b]==0:            #La célula central no se considera en el conteo  
        sumaS=sumaS-1         #Si la célula central es susceptible se elimina de la suma
    return sumaS              #cantidad de suscptibles en la vecindad V

def sumaI(V,a,b):             #Cantidad de individuos infectados en la vecindad V
    sumaI=0                   #Antes de realizar el conteo se toma un contador igual a cero
    n,m=V.shape
    for i in range(n):        #Tres filas en la vecindad de Moore
        for j in range(m):    #Tres columnas en la vecindad de Moore
            if V[i][j]==1:    #Para cada individuo se verifica si está identificado con el estado I
                sumaI=sumaI+1 #Si es verdadero, aumente en uno el contador de infectados
    if V[a][b]==1:            #La célula central no se considera en el conteo  
        sumaI=sumaI-1         #Si la célula central esta infectada se elimina de la suma
    return sumaI              #cantidad de infectados en la vecindad V

def sumaR(V,a,b):             #Cantidad de individuos recuperados en la vecindad V
    sumaR=0                   #Antes de realizar el conteo se toma un contador igual a cero
    n,m=V.shape
    for i in range(n):        #Tres filas en la vecindad de Moore
        for j in range(m):    #Tres columnas en la vecindad de Moore
            if V[i][j]==2:    #Para cada individuo se verifica si está identificado con el estado R
                sumaR=sumaR+1 #Si es verdadero, aumente en uno el contador de recuperados
    if V[a][b]==2:            #La célula central no se considera en el conteo  
        sumaR=sumaR-1         #Si la célula central esta recuperada se elimina de la suma
    return sumaR              #cantidad de recuperados en la vecindad V

def sumaV(V,a,b):             #Cantidad de espacios vacios en la vecindad V
    sumaV=0                   #Antes de realizar el conteo se toma un contador igual a cero
    n,m=V.shape
    for i in range(n):        #Tres filas en la vecindad de Moore
        for j in range(m):    #Tres columnas en la vecindad de Moore
            if V[i][j]==-1:   #Para cada espacio se verifica si está identificado con el estado V
                sumaV=sumaV+1 #Si es verdadero, aumente en uno el contador de recuperados
    if V[a][b]==-1:           #La célula central no se considera en el conteo  
        sumaV=sumaV-1         #Si la célula central no está se elimina de la suma
    return sumaV              #cantidad de espacios vacios en la vecindad V

In [ ]:


#---Sistema----

def count_S(A):             #Cantidad de individuos que poseen el estado S en el arreglo A
    n,m=A.shape             #Dim(A)=nm
    suma=0                  #Antes de realizar el conteo se toma un contador igual a cero
    for i in range(n):      #n filas del arreglo A
        for j in range(m):  #m columnas del arreglo A
            if A[i][j]==0:  #Para cada espacio se verifica si está identificado con el estado S
                suma=suma+1 #Si es verdadero, aumente en uno el contador de susceptibles
    return int(suma)        #cantidad de individuos susceptibles en el arreglo A

def count_I(A):             #Cantidad de individuos que poseen el estado I en el arreglo A
    n,m=A.shape             #Dim(A)=nm
    suma=0                  #Antes de realizar el conteo se toma un contador igual a cero
    for i in range(n):      #n filas del arreglo A
        for j in range(m):  #m columnas del arreglo A
            if A[i][j]==1:  #Para cada espacio se verifica si está identificado con el estado I
                suma=suma+1 #Si es verdadero, aumente en uno el contador de infectados
    return int(suma)        #cantidad de individuos infectados en el arreglo A

def count_R(A):             #Cantidad de individuos que poseen el estado R en el arreglo A
    n,m=A.shape             #Dim(A)=nm
    suma=0                  #Antes de realizar el conteo se toma un contador igual a cero
    for i in range(n):      #n filas del arreglo A
        for j in range(m):  #m columnas del arreglo A
            if A[i][j]==2:  #Para cada espacio se verifica si está identificado con el estado R
                suma=suma+1 #Si es verdadero, aumente en uno el contador de recuperados
    return int(suma)        #cantidad de individuos recuperados en el arreglo A

def count_D(A):             #Cantidad de individuos que poseen el estado D en el arreglo A
    n,m=A.shape             #Dim(A)=nm
    suma=0                  #Antes de realizar el conteo se toma un contador igual a cero
    for i in range(n):      #n filas del arreglo A
        for j in range(m):  #m columnas del arreglo A
            if A[i][j]==3:  #Para cada espacio se verifica si está identificado con el estado D
                suma=suma+1 #Si es verdadero, aumente en uno el contador de recuperados
    return int(suma)        #cantidad de individuos muertos en el arreglo A

def num_individuals(A): #Función contadora de individuos que interactuan en el sistema A 
    n = count_S(A)      #Cantidad de individuos que poseen el estado S
    m = count_I(A)      #Cantidad de individuos que poseen el estado I
    o = count_R(A)      #Cantidad de individuos que poseen el estado R
    d = count_D(A)      #Cantidad de individuos que poseen el estado D
    ci = n + m + o + d  #Total de individuos que interactuan en el sistema
    return ci           #Retorna ci

#-----Funciones de normalización------

def count_s(A):                          #Función de población normalizada para el estado S          
    return count_S(A)/num_individuals(A) #Promedio de susceptibles con respecto a células que interactuan

def count_i(A):                          #Función de población normalizada para el estado I
    return count_I(A)/num_individuals(A) #Promedio de infectados con respecto a células que interactuan

def count_r(A):                          #Función de población normalizada para el estado R
    return count_R(A)/num_individuals(A) #Promedio de recuperados con respecto a células que interactuan

def count_d(A):                          #Función de población normalizada para el estado D
    return count_D(A)/num_individuals(A) #Promedio de muertos con respecto a células que interactuan

In [ ]:
def base_rule(alpha,beta,V,a,b):                                  #Regla base de interacción local      
    I = sumaI(V,a,b); S = sumaS(V,a,b); Va = sumaV(V,a,b)         #Se realiza un conteo de infectados, susceptibles y espacios vacios en la vecindad V            
    n,m=V.shape
    B = np.zeros((n,m))                                           #Para evitar problemas de apuntadores se realiza la transformación sobre una copia de V
    r = random.randint(0,100)                                     #El valor aleatorio rho
    for i in range(n):                                            #Copia de las tres filas de la vecindad V
        for j in range(m):                                        #Copia de las tres columnas de la vecindad V
            B[i][j] = V[i][j]                                     #Copia de los indiviuos en la vecindad V
    if V[1][1] != -1:                                             #Se consideran unicamente los espacios no vacios ya que la tranformación de un espacio vacio es vacio
        if I > 0:                                                 #Si no hay infectados en la vecindad, no hay propagacióón de la enfermedad en la vecindad
            if I <= S and r >= (I/((n*m-1)-Va))*(beta/alpha)*100: #Condición para transición al estado S 
                B[a][b] = 0     
            else:                                                 #Condición para transición al estado I
                B[a][b] = 1
        else:
            B[a][b] = V[a][b]
    else:                                       #Si la célula es vacia se mantiene vacia
        B[a][b] = -1
    return B[a][b]                              #Al ser una regla totalíística, se retorna el valor central

In [ ]:
#-----Modelo SIS----

def evolution_sis(alpha,beta,U,p,q,function): #Regla base de evolucion en U de parametros alpha y beta (p y q = tamaño extra para las esquinas) text = Vecindad
    n,m=U.shape                           #dim(U)=nm
    B=np.zeros((n+(p*2),m+(q*2)))          #B será un arreglo nulo sobre el cual se sobreescribiran los datos de U
    C=np.zeros((n,m))                     #C será el arreglo que guardará los datos luego de aplicar la regla base de evolución
    I=count_I(U)                          #Se aplicará la regla siempre y cuando existan individuos infectados
    for i in range(n):                              
        for j in range(m):
            B[i+p][j+q]=U[i][j]   #Se sobreescriben los datos de U sobre B
    for i in range(n):
        for j in range(m):
            if I > 0:                                                      #Si hay individuos infectados en el sistema
                V = identificador(function,B,i+p,j+q)
                C[i][j]=base_rule(alpha,beta,V[0],V[1][0][0],V[1][0][1])      #Aplique la regla base de evolución local a U y guarde los valores en C 
            else:                                                          #Si no hay individuos infectados
                C[i][j] = U[i][j]                                          #La regla base de evolución actua como la función identica sobre el sistema
    return C                      #Retorne la evolución del sistema

def evolution_SIS(alpha,beta,tf,A,p,q,function):          #Definición del conjunto de evoluciones 
    L = [A]                                           #El primer elemento del conjunto es la configuracióón inicial
    i = 0                                             #Se define el contador de composiciones
    while i <= tf:                                    #Se realizarán composiciones hasta el tiempo tf 
        i = i + 1                                               #El contador crece en una unidad
        L.append(evolution_sis(alpha,beta,L[i-1],p,q,function))     #Se añade la composición i del arreglo en la posición i-1 
    return L                                                    #Se retorna el conjunto de composiciones que describe la evolucióón de A

def SIS_model(alpha,beta,tf,A,p,q,function):          #Modelo SIS
    S = []; I = []                                #S e I guardan la cantidad de individuos pertenecientes a los estados S e I por cada iteración 
    CI = np.zeros((tf,2)); CS = np.zeros((tf,2))  #CS y CI nos permitiran graficar los datos de S e I
    B = evolution_SIS(alpha,beta,tf,A,p,q,function)   #Calculamos la evolución inicial del sistema A hasta el tiempo tf
    for j in range(tf):                           #Para cada iteración defina
        M = B[j]                                  #M será la evolución en el instante t=i
        S.append(count_s(M))                      #Añadimos el promedio de susceptibles en M a S 
        I.append(count_i(M))                      #Añadimos el promedio de infectados en M a I                           
    for i in range(tf):                           #Para cada instante i hasta tf defina
        CS[i][0] = i; CS[i][1] = S[i]             #CS[i]=(i,S[i]) -> (tiempo i, cantidad de susceptibles en el tiempo i)
        CI[i][0] = i; CI[i][1] = I[i]             #CI[i]=(i,I[i]) -> (tiempo i, cantidad de infectados en el tiempo i)
    return [CS,CI,S,I,B]                          #Lista con las parejas ordenadas CS, CI, cantidad de individuos susceptibles, infectados y las evoluciones del sistema